In [41]:
import os
os.environ['SPARK_HOME'] = "/home/zaranik/.sdkman/candidates/spark/current"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python3'

In [42]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import Column

In [43]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
spark = SparkSession.builder.appName("lab1-frogs").getOrCreate()
# Define the schema for the data
frog_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("pres.abs", IntegerType(), False),
    StructField("northing", IntegerType(), False),
    StructField("easting", IntegerType(), False),
    StructField("altitude", IntegerType(), False),
    StructField("distance", IntegerType(), False),
    StructField("NoOfPools", IntegerType(), False),
    StructField("NoOfSites", IntegerType(), False),
    StructField("avrain", DoubleType(), False),
    StructField("meanmin", DoubleType(), False),
    StructField("meanmax", DoubleType(), False)
])

frogs_df = spark.read.csv("./frogs.csv", header=True, schema=frog_schema)
frogs_df = frogs_df.withColumnRenamed("pres.abs", "presence")
frogs_df.show()

+---+--------+--------+-------+--------+--------+---------+---------+-----------+----------------+----------------+
| id|presence|northing|easting|altitude|distance|NoOfPools|NoOfSites|     avrain|         meanmin|         meanmax|
+---+--------+--------+-------+--------+--------+---------+---------+-----------+----------------+----------------+
|  2|       1|     115|   1047|    1500|     500|      232|        3|      155.0|3.56666666666667|            14.0|
|  3|       1|     110|   1042|    1520|     250|       66|        5|157.6666667|3.46666666666667|            13.8|
|  4|       1|     112|   1040|    1540|     250|       32|        5|159.6666667|             3.4|            13.6|
|  5|       1|     109|   1033|    1590|     250|        9|        5|      165.0|             3.2|13.1666666666667|
|  6|       1|     109|   1032|    1590|     250|       67|        5|      165.0|             3.2|13.1666666666667|
|  7|       1|     106|   1018|    1600|     500|       12|        4|167

24/10/01 21:29:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , pres.abs, northing, easting, altitude, distance, NoOfPools, NoOfSites, avrain, meanmin, meanmax
 Schema: id, pres.abs, northing, easting, altitude, distance, NoOfPools, NoOfSites, avrain, meanmin, meanmax
Expected: id but found: 
CSV file: file:///home/zaranik/spark-tutorial/tutorial/labs/frogs.csv


In [44]:
df = frogs_df.filter(frogs_df["presence"] == 1).agg({"altitude": "max"})
df.show()

+-------------+
|max(altitude)|
+-------------+
|         1700|
+-------------+



2. Вивести всі висоти, де середні максимальні температури не
перевищують 14 градусів і на яких спостерігались жаби.

In [45]:
df = frogs_df.filter(frogs_df["presence"] == 1).filter(frogs_df["meanmax"] < 14).orderBy(frogs_df["meanmax"].desc())
df.select("altitude", "meanmax").show()

+--------+----------------+
|altitude|         meanmax|
+--------+----------------+
|    1510|13.8666666666667|
|    1520|13.8666666666667|
|    1520|            13.8|
|    1520|            13.8|
|    1530|            13.8|
|    1520|13.7666666666667|
|    1540|13.7333333333333|
|    1540|13.6666666666667|
|    1540|            13.6|
|    1560|13.5333333333333|
|    1560|13.5333333333333|
|    1560|13.4333333333333|
|    1560|13.4333333333333|
|    1560|            13.4|
|    1580|            13.4|
|    1580|            13.3|
|    1580|13.2666666666667|
|    1600|            13.2|
|    1600|            13.2|
|    1590|13.1666666666667|
+--------+----------------+
only showing top 20 rows

